In [ ]:
N = 3
c=2
t_steps = c*N
order = 13
k = 2
D = 2

In [ ]:
import netgen.meshing as ngm
from netgen.geom2d import unit_square
from ngsolve import *

ngmesh = ngm.Mesh()
ngmesh.SetGeometry(unit_square)
ngmesh.dim = 2
pnums = []
for i in range(t_steps + 1):
    for j in range(N + 1):
        pnums.append(ngmesh.Add(ngm.MeshPoint(ngm.Pnt(i / t_steps, j / N, 0))))

foo = ngm.FaceDescriptor(surfnr=1,domin=1,bc=1)
ngmesh.Add (foo)
ngmesh.SetMaterial(1, "mat")
for j in range(t_steps):
    for i in range(N):
        ngmesh.Add(ngm.Element2D(1, [pnums[i + j * (N + 1)],
                                 pnums[i + (j + 1) * (N + 1)],
                                 pnums[i + 1 + (j + 1) * (N + 1)],
                                 pnums[i + 1 + j * (N + 1)]]))
for i in range(t_steps):
   ngmesh.Add(ngm.Element1D([pnums[N + i * (N + 1)], pnums[N + (i + 1) * (N + 1)]], index=1))
   ngmesh.Add(ngm.Element1D([pnums[0 + i * (N + 1)], pnums[0 + (i + 1) * (N + 1)]], index=1))
for i in range(N):
   ngmesh.Add(ngm.Element1D([pnums[i], pnums[i + 1]], index=2))
   ngmesh.Add(ngm.Element1D([pnums[i + t_steps * (N + 1)], pnums[i + 1 + t_steps * (N + 1)]], index=2))

mesh = Mesh(ngmesh)
Draw(mesh)
# print("boundaries" + str(mesh.GetBoundaries()))

In [ ]:
from ngsolve import *
from trefftzngs import *
import numpy as np

truesol =  sin( k*(c*x + y) )#exp(-pow(c*x+y,2)))#
v0 = c*k*cos(k*(c*x+y))#grad(U0)[0]
sig0 = -k*cos(k*(c*x+y))#-grad(U0)[1]


In [ ]:
def MakeSystem(truesol,v0,sig0,fes):
    U = fes.TrialFunction()
    V = fes.TestFunction()

    v = grad(U)[0]
    sig = -grad(U)[1]
    w = grad(V)[0]
    tau = -grad(V)[1]

    vo = grad(U.Other())[0]
    sigo = -grad(U.Other())[1]
    wo = grad(V.Other())[0]
    tauo = -grad(V.Other())[1]

    h = specialcf.mesh_size
    n = specialcf.normal(2)
    n_t = n[0]/Norm(n)
    n_x = n[1]/Norm(n)

    mean_v = 0.5*(v+vo)
    mean_w = 0.5*(w+wo)
    mean_sig = 0.5*(sig+sigo)
    mean_tau = 0.5*(tau+tauo)

    jump_vx = ( v - vo ) * n_x
    jump_wx = ( w - wo ) * n_x
    jump_sigx = ( sig - sigo ) * n_x
    jump_taux = ( tau - tauo ) * n_x

    jump_vt = ( v - vo ) * n_t
    jump_wt = ( w - wo ) * n_t
    jump_sigt = ( sig - sigo ) * n_t
    jump_taut = ( tau - tauo ) * n_t

    jump_Ut = (U - U.Other()) * n_t

    timelike = n_x**2 #IfPos(n_t,0,IfPos(-n_t,0,1)) # n_t=0
    spacelike = n_t**2 #IfPos(n_x,0,IfPos(-n_x,0,1)) # n_x=0

    alpha = 0.5 
    beta = 0.5 
    gamma = 1

    a = BilinearForm(fes)
    
    # a += SymbolicBFI( spacelike * ( IfPos(n_t,v,vo)*(pow(c,-2)*jump_wt+jump_taux) + IfPos(n_t,sig,sigo)*(jump_wx+jump_taut) ) ,VOL,  skeleton=True ) #space like faces
    a += SymbolicBFI( spacelike * ( IfPos(n_t,v,vo)*(pow(c,-2)*jump_wt) + IfPos(n_t,sig,sigo)*(jump_taut) ) ,VOL,  skeleton=True ) #space like faces, no jump in x since horizontal
    a += SymbolicBFI( timelike 	* ( mean_v*jump_taux + mean_sig*jump_wx + alpha*jump_vx*jump_wx + beta*jump_sigx*jump_taux ) ,VOL, skeleton=True ) #time like faces
    a += SymbolicBFI( spacelike * IfPos(n_t,1,0) * ( pow(c,-2)*v*w + sig*tau ), BND, skeleton=True) #t=T (or *x)
    a += SymbolicBFI( timelike 	* ( sig*n_x*w + alpha*v*w ), BND, skeleton=True) #dirichlet boundary 'timelike'
    a += SymbolicBFI( spacelike * ( gamma * (jump_Ut)*IfPos(n_t,V.Other(),V) ) ,VOL,  skeleton=True ) #correction term to recover sol of second order system
    a += SymbolicBFI( spacelike * ( gamma * IfPos(-n_t,1,0) * U*V ) ,BND,  skeleton=True ) #BND correction term to recover sol of second order system
    a.Assemble()

    f = LinearForm(fes)
    f += SymbolicLFI( spacelike * IfPos(-n_t,1,0) *  ( pow(c,-2)*v0*w + sig0*tau ), BND, skeleton=True) #t=0 (or *(1-x))
    f += SymbolicLFI( timelike 	* ( v0 * (alpha*w - tau*n_x) ), BND, skeleton=True) #dirichlet boundary 'timelike'
    f += SymbolicLFI( spacelike * gamma * IfPos(-n_t,1,0) *  ( (truesol)*V ) ,BND,  skeleton=True ) #rhs correction term to recover sol of second order system
    f.Assemble()

    return [a,f]
    # gfu2= GridFunction(fes, name="uDG")
    # gfu2.vec.data = a.mat.Inverse() * f.vec

In [ ]:
def MakeL2System(v0,sig0,fes):
    v,sig = fes.TrialFunction()
    w,tau = fes.TestFunction()

    vo = v.Other()
    sigo = sig.Other()
    wo = w.Other()
    tauo = tau.Other()

    h = specialcf.mesh_size
    n = specialcf.normal(2)
    n_t = n[0]/Norm(n)
    n_x = n[1]/Norm(n)

    mean_v = 0.5*(v+vo)
    mean_w = 0.5*(w+wo)
    mean_sig = 0.5*(sig+sigo)
    mean_tau = 0.5*(tau+tauo)

    jump_vx = ( v - vo ) * n_x
    jump_wx = ( w - wo ) * n_x
    jump_sigx = ( sig - sigo ) * n_x
    jump_taux = ( tau - tauo ) * n_x

    jump_vt = ( v - vo ) * n_t
    jump_wt = ( w - wo ) * n_t
    jump_sigt = ( sig - sigo ) * n_t
    jump_taut = ( tau - tauo ) * n_t

    timelike = n_x**2 #IfPos(n_t,0,IfPos(-n_t,0,1)) # n_t=0
    spacelike = n_t**2 #IfPos(n_x,0,IfPos(-n_x,0,1)) # n_x=0

    alpha = 0.5 #pow(10,5)
    beta = 0.5 #pow(10,5)
    gamma = 1

    a = BilinearForm(fes)
    a += SymbolicBFI(  -v*(grad(tau)[1]+pow(c,-2)*grad(w)[0]) - sig*(grad(tau)[0]+grad(w)[1])  )
    a += SymbolicBFI( spacelike * ( IfPos(n_t,v,vo)*(pow(c,-2)*jump_wt) + IfPos(n_t,sig,sigo)*(jump_taut) ) ,VOL,  skeleton=True ) #space like faces, no jump in x since horizontal
    a += SymbolicBFI( timelike 	* ( mean_v*jump_taux + mean_sig*jump_wx + alpha*jump_vx*jump_wx + beta*jump_sigx*jump_taux ) ,VOL, skeleton=True ) #time like faces
    a += SymbolicBFI( spacelike * IfPos(n_t,1,0) * ( pow(c,-2)*v*w + sig*tau ), BND, skeleton=True) #t=T (or *x)
    a += SymbolicBFI( timelike 	* ( sig*n_x*w + alpha*v*w ), BND, skeleton=True) #dirichlet boundary 'timelike'
    a.Assemble()

    f = LinearForm(fes)
    f += SymbolicLFI( spacelike * IfPos(-n_t,1,0) *  ( pow(c,-2)*v0*w + sig0*tau ), BND, skeleton=True) #t=0 (or *(1-x))
    f += SymbolicLFI( timelike 	* ( v0 * (alpha*w - tau*n_x) ), BND, skeleton=True) #dirichlet boundary 'timelike'
    f.Assemble()

    return [a,f]

In [ ]:
def SolveNp(a,f,fes):
	gfu = GridFunction(fes, name="uDG")

	nmat = np.zeros((a.mat.height,a.mat.width))
	nvec = np.zeros(a.mat.width)

	for i in range(a.mat.width):#gfu.vec.data = a.mat.Inverse() * f.vec
		for j in range(a.mat.height):
			nmat[j,i] = a.mat[j,i]
	nvec = f.vec.FV().NumPy() #nvec

	sol = np.linalg.solve(nmat,nvec)
	for i in range(a.mat.height):
		gfu.vec[i] = sol[i]
	# print("cond nmat: ", np.linalg.cond(nmat))
	return [gfu,np.linalg.cond(nmat)]

In [ ]:
def SolveTrefftz(order,btype,c,truesol,v0,sig0):
    fes = FESpace("trefftzfespace", mesh, order = order, wavespeed = c, dgjumps=True, basistype = btype)
    [a,f] = MakeSystem(truesol,v0,sig0,fes)
    [gfu,cond] = SolveNp(a,f,fes)
    
    dof = fes.ndof/(N*t_steps)#scipy.special.binom(D-1 + ordr, ordr) + scipy.special.binom(D-1 + ordr-1, ordr-1)

    L2error = sqrt(Integrate((truesol - gfu)*(truesol - gfu), mesh))
    U0 = GridFunction(fes)
    U0.Set(truesol)
    sH1error = sqrt(Integrate((grad(U0) - grad(gfu))*(grad(U0) - grad(gfu)), mesh))
    return [dof,cond,L2error,sH1error]

def SolveL2(order,truesol,v0,sig0):
    X = L2(mesh, order=order)
    fes = FESpace([X,X], flags = { "dgjumps" : True })
    [a,f] = MakeL2System(v0,sig0,fes)
    [gfu,cond] = SolveNp(a,f,fes)
    
    dof=fes.ndof/(N*t_steps)#X.ndof/(N*t_steps)
    
    L2error=1
    U0 = GridFunction(fes)
    U0.components[0].Set(v0)
    U0.components[1].Set(sig0)
    err0=U0.components[0] - gfu.components[0]
    err1=U0.components[1] - gfu.components[1]
    sH1error = sqrt(Integrate( err0*err0 + err1*err1, mesh))
    return [dof,cond,L2error,sH1error]
    

In [ ]:
import scipy.special
solution = []
for ordr in range(5,order):
    print("run order: " + str(ordr))
    
    btype = 0
    [dof,cond,L2error,sH1error] = SolveL2(ordr-4,truesol,v0,sig0)
    solution.append([btype, ordr-4, dof,cond,L2error,sH1error])
    print("btype: " + str(btype) + " dof: " + str(dof) + " cond: " + str(cond) + " L2error: " + str(L2error) + " H1error: "+ str(sH1error))
    
    btype = 1
    [dof,cond,L2error,sH1error] = SolveTrefftz(ordr,0,c,truesol,v0,sig0)
    solution.append([btype,ordr,dof,cond,L2error,sH1error])
    print("btype: " + str(btype) + " dof: " + str(dof) + " cond: " + str(cond) + " L2error: " + str(L2error) + " H1error: "+ str(sH1error))
      
    

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

def errfig(sol,xaxis,yaxis,xlabel,saveas):
    fig = plt.figure()
    ax = fig.gca()
    plt.semilogy(sol[sol[:,0]==0,xaxis], sol[sol[:,0]==0,yaxis], '-o', label="L2")
    plt.semilogy(sol[sol[:,0]==1,xaxis], sol[sol[:,0]==1,yaxis], '--x', label="Trefftz")
    #plt.semilogy(sol[sol[:,0]==2,xaxis], sol[sol[:,0]==2,yaxis], '--s', label="Chebyshev")
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    plt.title('Wavespeed: ' + str(c) + " Meshsize: " + str(1/N))
    plt.legend()
    plt.ylabel(data[i])
    plt.xlabel(xlabel)
    plt.savefig(saveas)
    plt.show()

data = ['condition', 'L2error', 'H1error']
sol = np.array(solution)

for i in [0,2]:
    errfig(sol,1,i+3,'order',"results/as_"+data[i]+".png")
    errfig(sol,2,i+3,'dof',"results/asdof_"+data[i]+".png")